In [1]:
# if kernel crashes, make sure pywin32 and pipywin32 are installed. 
# Followed instructions here: https://github.com/jupyter/notebook/issues/4909 
import win32api

In [3]:

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import microdf as mdf
import os
import us



In [4]:
# Import data from Ipums
person_raw = pd.read_csv("cps_00041.csv.gz")

# Create copy and lower column names
person = person_raw.copy(deep=True)
person.columns = person.columns.str.lower()
person.asecwt /= 3

# Crate booleans for demographics
person["adult"] = person.age > 17
person["child"] = person.age < 18

person["black"] = person.race == 200
person["white_non_hispanic"] = (person.race == 100) & (person.hispan == 0)
person["hispanic"] = (person.hispan > 1) & person.hispan < 700
person["pwd"] = person.diffany == 2
person["non_citizen"] = person.citizen == 5
person["non_citizen_child"] = (person.citizen == 5) & (person.age < 18)
person["non_citizen_adult"] = (person.citizen == 5) & (person.age > 17)

# Remove NIUs
person["taxinc"].replace({9999999: 0}, inplace=True)
person["adjginc"].replace({99999999: 0}, inplace=True)
person["incss"].replace({999999: 0}, inplace=True)
person["incssi"].replace({999999: 0}, inplace=True)
person["incunemp"].replace({99999: 0}, inplace=True)
person["incunemp"].replace({999999: 0}, inplace=True)
person["ctccrd"].replace({999999: 0}, inplace=True)
person["actccrd"].replace({99999: 0}, inplace=True)
person["eitcred"].replace({9999: 0}, inplace=True)
person["fica"].replace({99999: 0}, inplace=True)
person["fedtaxac"].replace({99999999: 0}, inplace=True)
person["stataxac"].replace({9999999: 0}, inplace=True)

In [5]:
# Change fip codes to state names
person["statefip"]=person["statefip"].astype(str)
person["statefip"]=person["statefip"].apply(lambda x: us.states.lookup(x))

# Aggregate deductible and refundable child tax credits
person["ctc"] = person.ctccrd + person.actccrd

# Calculate the number of people per smp unit
person["person"] = 1
spm = person.groupby(["spmfamunit", "year"])[["person"]].sum()
spm.columns = ["numper"]
person = person.merge(spm, left_on=["spmfamunit", "year"], right_index=True)

person["weighted_state_tax"] = person.asecwt * person.stataxac
person["weighted_agi"] = person.asecwt * person.adjginc

# Calculate the total taxable income and total people in each state
state_groups_taxinc = person.groupby(["statefip"])[
    ["weighted_state_tax", "weighted_agi"]
].sum()
state_groups_taxinc.columns = ["state_tax_revenue", "state_taxable_income"]
person = person.merge(
    state_groups_taxinc, left_on=["statefip"], right_index=True
)


In [6]:
person.statefip.value_counts()

Texas                   32167
Florida                 23335
New York                20722
Illinois                14174
Pennsylvania            13043
Louisiana               11970
Ohio                    11969
Georgia                 11810
North Carolina          11806
Michigan                11196
Massachusetts           10470
New Jersey              10058
Washington              10019
New Mexico               9816
Virginia                 9716
West Virginia            9627
Tennessee                9510
Mississippi              9433
Utah                     9183
Montana                  8775
Idaho                    8573
Indiana                  8432
Hawaii                   8249
South Carolina           8231
Oklahoma                 8040
District of Columbia     7917
Oregon                   7760
Nevada                   7188
North Dakota             7071
Wyoming                  7018
Wisconsin                6849
Missouri                 6830
Minnesota                6721
Maryland  